## GAT training test

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.utils import from_networkx
import networkx as nx
from torch_geometric.nn import GATConv

In [8]:
# 1. 加载 Karate Club 数据集
G = nx.karate_club_graph()

# 为节点添加特征（这里使用节点的度作为特征）
for node in G.nodes:
    G.nodes[node]['feature'] = float(G.degree(node))

# 为节点添加标签（这里使用俱乐部分组作为标签）
# Karate Club 图中，节点 0 和 33 属于不同的组
labels = [0 if G.nodes[node]['club'] == 'Mr. Hi' else 1 for node in G.nodes]

# 2. 将 NetworkX 图转换为 PyG 的 Data 对象
data = from_networkx(G)

# 添加节点特征和标签
data.x = torch.tensor([[G.nodes[node]['feature']] for node in G.nodes], dtype=torch.float)
data.y = torch.tensor(labels, dtype=torch.long)

# 添加训练、验证和测试掩码
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

# 随机分配训练、验证和测试集
torch.manual_seed(42)  # 设置随机种子以确保可重复性
indices = torch.randperm(data.num_nodes)
data.train_mask[indices[:23]] = True  # 25 个节点作为训练集
data.val_mask[indices[23:28]] = True  # 5 个节点作为验证集
data.test_mask[indices[28:]] = True   # 4 个节点作为测试集

In [9]:
# 3. 定义 GAT 模型
class GAT(torch.nn.Module):
     def __init__(self, num_features, hidden_channels, num_classes, heads=8):
          super(GAT, self).__init__()
          # 第一层 GAT
          self.conv1 = GATConv(num_features, hidden_channels, heads=heads, dropout=0.6)
          # 第二层 GAT
          self.conv2 = GATConv(hidden_channels * heads, num_classes, heads=1, dropout=0.6)

     def forward(self, x, edge_index):
          # 第一层 GAT
          x = F.dropout(x, p=0.6, training=self.training)
          x = F.elu(self.conv1(x, edge_index))
          # 第二层 GAT
          x = F.dropout(x, p=0.6, training=self.training)
          x = self.conv2(x, edge_index)
          return F.log_softmax(x, dim=1)

In [10]:
# 4. 初始化模型和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT(
     num_features=1,          # 输入特征维度（每个节点只有一个特征：度）
     hidden_channels=8,       # 隐藏层维度
     num_classes=2,           # 输出类别数（两个组）
     heads=8                  # 注意力头数
).to(device)

# 将数据移动到设备
data = data.to(device)

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

# 5. 训练函数
def train():
     model.train()
     optimizer.zero_grad()
     out = model(data.x, data.edge_index)  # 前向传播
     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])  # 计算损失
     loss.backward()  # 反向传播
     optimizer.step()  # 更新参数
     return loss.item()

# 6. 测试函数
def test():
     model.eval()
     out = model(data.x, data.edge_index)
     pred = out.argmax(dim=1)  # 获取预测类别
     correct = pred[data.test_mask] == data.y[data.test_mask]  # 计算正确预测数
     acc = int(correct.sum()) / int(data.test_mask.sum())  # 计算准确率
     return acc

# 7. 训练和测试
for epoch in range(1, 201):
     loss = train()
     if epoch % 10 == 0:
          acc = test()
          print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')

Epoch: 010, Loss: 1.5883, Test Acc: 0.1667
Epoch: 020, Loss: 0.6044, Test Acc: 0.1667
Epoch: 030, Loss: 0.6899, Test Acc: 0.1667
Epoch: 040, Loss: 0.9501, Test Acc: 0.8333
Epoch: 050, Loss: 1.0601, Test Acc: 0.8333
Epoch: 060, Loss: 0.6902, Test Acc: 0.1667
Epoch: 070, Loss: 0.7848, Test Acc: 0.1667
Epoch: 080, Loss: 0.7158, Test Acc: 0.0000
Epoch: 090, Loss: 0.6514, Test Acc: 0.8333
Epoch: 100, Loss: 0.6666, Test Acc: 0.1667
Epoch: 110, Loss: 0.9351, Test Acc: 0.1667
Epoch: 120, Loss: 0.7712, Test Acc: 0.8333
Epoch: 130, Loss: 0.7822, Test Acc: 0.1667
Epoch: 140, Loss: 0.6457, Test Acc: 0.1667
Epoch: 150, Loss: 0.6809, Test Acc: 0.1667
Epoch: 160, Loss: 0.8148, Test Acc: 0.1667
Epoch: 170, Loss: 0.6798, Test Acc: 0.1667
Epoch: 180, Loss: 0.7487, Test Acc: 0.1667
Epoch: 190, Loss: 0.7904, Test Acc: 0.1667
Epoch: 200, Loss: 0.7713, Test Acc: 0.6667
